In [2]:
import torch

In [3]:
inputs=torch.tensor(
    [
     [0.43,0.15,0.89],#your
     [0.55,0.87,0.66],#journey
     [0.57,0.85,0.64],#starts
     [0.22,0.58,0.33],#with
     [0.77,0.25,0.10],#one
     [0.05,0.80,0.55],#step
    ]
)

In [4]:
w_q=torch.nn.Linear(inputs.shape[1],2,bias=False)
print(w_q)
w_k=torch.nn.Linear(inputs.shape[1],2,bias=False)
w_v=torch.nn.Linear(inputs.shape[1],2,bias=False)

Linear(in_features=3, out_features=2, bias=False)


In [5]:
queries=w_q(inputs)
keys=w_k(inputs)
values=w_v(inputs)

print(queries)

tensor([[-0.3124, -0.5196],
        [-0.6133, -0.4779],
        [-0.6110, -0.4712],
        [-0.3421, -0.2364],
        [-0.3979, -0.2197],
        [-0.3861, -0.3193]], grad_fn=<MmBackward0>)


In [6]:
attn_score=queries@keys.T
print(attn_score)

tensor([[-0.2088, -0.3265, -0.3210, -0.1877, -0.1310, -0.2528],
        [-0.2887, -0.3284, -0.3266, -0.1654, -0.2030, -0.1952],
        [-0.2866, -0.3244, -0.3227, -0.1630, -0.2018, -0.1918],
        [-0.1543, -0.1658, -0.1653, -0.0809, -0.1103, -0.0921],
        [-0.1671, -0.1610, -0.1613, -0.0734, -0.1227, -0.0765],
        [-0.1859, -0.2174, -0.2160, -0.1110, -0.1297, -0.1331]],
       grad_fn=<MmBackward0>)


In [7]:
attn_weight=torch.softmax(attn_score/keys.shape[1]**0.5,dim=1)
print(attn_weight)

tensor([[0.1699, 0.1564, 0.1570, 0.1725, 0.1795, 0.1647],
        [0.1621, 0.1576, 0.1578, 0.1769, 0.1723, 0.1732],
        [0.1621, 0.1578, 0.1580, 0.1769, 0.1721, 0.1733],
        [0.1636, 0.1622, 0.1623, 0.1723, 0.1687, 0.1709],
        [0.1619, 0.1626, 0.1626, 0.1730, 0.1671, 0.1727],
        [0.1642, 0.1606, 0.1608, 0.1731, 0.1709, 0.1704]],
       grad_fn=<SoftmaxBackward0>)


In [8]:
#use pytorch tril function to diagonalize
context_length=inputs.shape[0]
mask1=torch.tril(torch.ones(context_length,context_length))
print(mask1)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [9]:
#use element wise multiplication not matrix
masked_attn_score=attn_weight*mask1
print(masked_attn_score)

#but this attention score is not fully masked or optimal becaus the softmax normalization adds influence on its from the previous attn score so use upper negative infinity mask instead of zero

tensor([[0.1699, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1621, 0.1576, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1621, 0.1578, 0.1580, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1622, 0.1623, 0.1723, 0.0000, 0.0000],
        [0.1619, 0.1626, 0.1626, 0.1730, 0.1671, 0.0000],
        [0.1642, 0.1606, 0.1608, 0.1731, 0.1709, 0.1704]],
       grad_fn=<MulBackward0>)


In [10]:
#normalize the masked attention score  to sum up 1 in each row to get masked attn weight
row_sum=masked_attn_score.sum(dim=1,keepdim=True)
print(row_sum)
masked_attn_weight=masked_attn_score/row_sum
print(masked_attn_weight)

tensor([[0.1699],
        [0.3198],
        [0.4778],
        [0.6604],
        [0.8273],
        [1.0000]], grad_fn=<SumBackward1>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5070, 0.4930, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3392, 0.3302, 0.3306, 0.0000, 0.0000, 0.0000],
        [0.2477, 0.2457, 0.2458, 0.2609, 0.0000, 0.0000],
        [0.1957, 0.1966, 0.1965, 0.2091, 0.2020, 0.0000],
        [0.1642, 0.1606, 0.1608, 0.1731, 0.1709, 0.1704]],
       grad_fn=<DivBackward0>)


In [11]:
masked_context_vectore=masked_attn_weight@values
print(masked_context_vectore)

tensor([[ 0.4517, -0.2462],
        [ 0.3241, -0.0025],
        [ 0.2786,  0.0797],
        [ 0.2224,  0.1132],
        [ 0.1822,  0.1180],
        [ 0.1739,  0.1360]], grad_fn=<MmBackward0>)


# Negative infinity masking to avoid any influence of future attention

In [12]:
print(attn_score)

tensor([[-0.2088, -0.3265, -0.3210, -0.1877, -0.1310, -0.2528],
        [-0.2887, -0.3284, -0.3266, -0.1654, -0.2030, -0.1952],
        [-0.2866, -0.3244, -0.3227, -0.1630, -0.2018, -0.1918],
        [-0.1543, -0.1658, -0.1653, -0.0809, -0.1103, -0.0921],
        [-0.1671, -0.1610, -0.1613, -0.0734, -0.1227, -0.0765],
        [-0.1859, -0.2174, -0.2160, -0.1110, -0.1297, -0.1331]],
       grad_fn=<MmBackward0>)


In [13]:
mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
print(mask)
masked=attn_score.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])


tensor([[-0.2088,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.2887, -0.3284,    -inf,    -inf,    -inf,    -inf],
        [-0.2866, -0.3244, -0.3227,    -inf,    -inf,    -inf],
        [-0.1543, -0.1658, -0.1653, -0.0809,    -inf,    -inf],
        [-0.1671, -0.1610, -0.1613, -0.0734, -0.1227,    -inf],
        [-0.1859, -0.2174, -0.2160, -0.1110, -0.1297, -0.1331]],
       grad_fn=<MaskedFillBackward0>)


In [14]:
masked_attn_weight=torch.softmax(masked/keys.shape[1],dim=1)
print(masked_attn_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5050, 0.4950, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3375, 0.3311, 0.3314, 0.0000, 0.0000, 0.0000],
        [0.2484, 0.2470, 0.2470, 0.2577, 0.0000, 0.0000],
        [0.1970, 0.1976, 0.1976, 0.2064, 0.2014, 0.0000],
        [0.1649, 0.1624, 0.1625, 0.1712, 0.1696, 0.1693]],
       grad_fn=<SoftmaxBackward0>)


# Masking additional attention weight with Dropout

In [15]:
torch.manual_seed(123)
dropout=torch.nn.Dropout(0.5)
print(dropout)
ones=torch.ones(6,6)
print(ones)
print(dropout(ones))

Dropout(p=0.5, inplace=False)
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [16]:
attn_dropout=dropout(attn_weight)
print(attn_dropout)

tensor([[0.3399, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3157, 0.0000, 0.3445, 0.3464],
        [0.0000, 0.3156, 0.3159, 0.3537, 0.3441, 0.3466],
        [0.0000, 0.0000, 0.3246, 0.3445, 0.0000, 0.3418],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3342, 0.3453],
        [0.3284, 0.3212, 0.0000, 0.3463, 0.3417, 0.3409]],
       grad_fn=<MulBackward0>)


# Causal attention Class with Batches

In [17]:
batch=torch.stack((inputs,inputs),dim=0)
print(batch)
print(batch.shape)

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
torch.Size([2, 6, 3])


In [18]:
print(batch.transpose(1,2))#change row(1) with colmn(2)

tensor([[[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]],

        [[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]]])


In [29]:
class Causal_Attention(torch.nn.Module):
    def __init__(self,in_d,out_d,context_length,dropout_percent):
        super().__init__()
        self.w_q=torch.nn.Linear(in_d,out_d)
        self.w_k=torch.nn.Linear(in_d,out_d)
        self.w_v=torch.nn.Linear(in_d,out_d)
        self.dropout=torch.nn.Dropout(dropout_percent)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    
    def forward(self,input_embedding):
        
        b,num_token,d_in=input_embedding.shape
        queries=self.w_q(input_embedding)
        keys=self.w_k(input_embedding)
        values=self.w_v(input_embedding)

        print(keys.shape[-1])

        attn_score=queries@keys.transpose(1,2)

        attn_score.masked_fill_(self.mask.bool()[:num_token, :num_token],-torch.inf)
        
        print('attn_score',attn_score)
        attn_weight=torch.softmax(attn_score/keys.shape[-1]**0.5,dim=-1)
        print('attn_weight',attn_weight)
        attn_weight=self.dropout(attn_weight)

        masked_context_vectore=attn_weight@values
        return masked_context_vectore

        
        

        
        


In [26]:
in_d,out_d=3,3
print(in_d)

3


In [30]:
torch.manual_seed(123)
context_length=batch.shape[1]
print(context_length)
ca=Causal_Attention(in_d,out_d,context_length,0.0)
print(ca)

ca.forward(batch)


6
Causal_Attention(
  (w_q): Linear(in_features=3, out_features=3, bias=True)
  (w_k): Linear(in_features=3, out_features=3, bias=True)
  (w_v): Linear(in_features=3, out_features=3, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)
3
attn_score tensor([[[-0.2173,    -inf,    -inf,    -inf,    -inf,    -inf],
         [-0.6607, -0.5641,    -inf,    -inf,    -inf,    -inf],
         [-0.6516, -0.5573, -0.5477,    -inf,    -inf,    -inf],
         [-0.3954, -0.3047, -0.3003, -0.2375,    -inf,    -inf],
         [-0.2972, -0.2652, -0.2612, -0.2145, -0.1605,    -inf],
         [-0.5065, -0.3916, -0.3856, -0.3170, -0.2411, -0.3807]],

        [[-0.2173,    -inf,    -inf,    -inf,    -inf,    -inf],
         [-0.6607, -0.5641,    -inf,    -inf,    -inf,    -inf],
         [-0.6516, -0.5573, -0.5477,    -inf,    -inf,    -inf],
         [-0.3954, -0.3047, -0.3003, -0.2375,    -inf,    -inf],
         [-0.2972, -0.2652, -0.2612, -0.2145, -0.1605,    -inf],
         [-0.5065, -0.3916, -0.

tensor([[[-0.0755,  0.2528,  0.4010],
         [ 0.0065,  0.5118,  0.5036],
         [ 0.0316,  0.5984,  0.5364],
         [ 0.0758,  0.6148,  0.4682],
         [ 0.0707,  0.6636,  0.4533],
         [ 0.0984,  0.6533,  0.4289]],

        [[-0.0755,  0.2528,  0.4010],
         [ 0.0065,  0.5118,  0.5036],
         [ 0.0316,  0.5984,  0.5364],
         [ 0.0758,  0.6148,  0.4682],
         [ 0.0707,  0.6636,  0.4533],
         [ 0.0984,  0.6533,  0.4289]]], grad_fn=<UnsafeViewBackward0>)

# Multi head attention wrapper class

In [33]:
class MultiHeadAttentionWrapper(torch.nn.Module):

    def __init__(self,in_d,out_d,context_length,dropout_percent,num_head):

        super().__init__()
        self.heads=torch.nn.ModuleList([Causal_Attention(in_d,out_d,context_length,dropout_percent) for _ in range(num_head)])
        print('self head',self.heads)
    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads],dim=-1)

In [34]:
torch.manual_seed(123)
context_length=batch.shape[1]
print(context_length)
mha=MultiHeadAttentionWrapper(in_d,out_d,context_length,0.0,num_head=2)

context_vec=mha(batch)
print('context_veccccccccc',context_vec)




6
self head ModuleList(
  (0-1): 2 x Causal_Attention(
    (w_q): Linear(in_features=3, out_features=3, bias=True)
    (w_k): Linear(in_features=3, out_features=3, bias=True)
    (w_v): Linear(in_features=3, out_features=3, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
)
3
attn_score tensor([[[-0.2173,    -inf,    -inf,    -inf,    -inf,    -inf],
         [-0.6607, -0.5641,    -inf,    -inf,    -inf,    -inf],
         [-0.6516, -0.5573, -0.5477,    -inf,    -inf,    -inf],
         [-0.3954, -0.3047, -0.3003, -0.2375,    -inf,    -inf],
         [-0.2972, -0.2652, -0.2612, -0.2145, -0.1605,    -inf],
         [-0.5065, -0.3916, -0.3856, -0.3170, -0.2411, -0.3807]],

        [[-0.2173,    -inf,    -inf,    -inf,    -inf,    -inf],
         [-0.6607, -0.5641,    -inf,    -inf,    -inf,    -inf],
         [-0.6516, -0.5573, -0.5477,    -inf,    -inf,    -inf],
         [-0.3954, -0.3047, -0.3003, -0.2375,    -inf,    -inf],
         [-0.2972, -0.2652, -0.2612, -0.2145, -0.

# MultiHead attention with weight splits